# Исследование корреляции солнечной активности и биржи

In [ ]:
import pandas as pd

In [ ]:
#url = 'https://www.spaceweather.gc.ca/solar_flux_data/daily_flux_values/fluxtable.txt'
df = pd.read_csv('fluxtable.txt', delim_whitespace=True, skiprows=[1])

# Объединяем fluxdate и fluxtime в одно поле timestamp
df['timestamp'] = pd.to_datetime(df['fluxdate'].astype(str) + df['fluxtime'].astype(str), format='%Y%m%d%H%M%S')

# Удаляем старые колонки fluxdate и fluxtime
df.drop(columns=['fluxdate', 'fluxtime'], inplace=True)

# Перемещаем колонку timestamp в начало
df = df[['timestamp'] + [col for col in df.columns if col != 'timestamp']]
df.head(3)

In [ ]:
solar=df.set_index('timestamp')[['fluxadjflux']].resample('D').mean().reset_index()
solar.head(3)

In [ ]:
import yfinance as yf

In [ ]:
tickers = '''AAPL, MSFT, AMZN, GOOGL, META, TSLA, JNJ, JPM, V, PG, NVDA, WMT, 
             DIS, BAC, MA, INTC, PFE, XOM, NFLX, ADBE, CRM, KO, PEP, GE'''

# Загружаем данные с конкретной начальной даты
raw_data = yf.download(
    tickers=tickers,
    start="2004-10-28",
    interval="1d",
    progress=True,
    group_by='ticker',
    threads=True ,
)

In [ ]:
raw_data.head(3)

In [ ]:
# Извлекаем цены закрытия и преобразуем структуру
close_prices = (
    raw_data
    .xs('Close', level=1, axis=1)      # Выбираем уровень Close
    .sort_index()                       # Сортируем по времени
)
close_prices.head(3)

In [ ]:
# Для солнечных данных
solar = df.set_index('timestamp')[['fluxadjflux']].resample('D').mean()

# Для цен акций (ваш текущий код)
close_prices = raw_data.xs('Close', level=1, axis=1).sort_index()

# Объединяем данные в один DataFrame
combined = solar.join(close_prices, how='inner').dropna()


combined.head(5)

In [ ]:
# Рассчитываем корреляции с статистической значимостью
from scipy.stats import pearsonr
ticker_columns = [col for col in combined.columns if col != 'fluxadjflux']

corr_list = []
for ticker in ticker_columns:
    r, p_value = pearsonr(combined['fluxadjflux'], combined[ticker])
    corr_list.append({
        'Ticker': ticker,
        'Correlation': r,
        'p-value': p_value,
        'Significant (5%)': p_value < 0.05
    })

corr_df = pd.DataFrame(corr_list).sort_values('Correlation', key=abs, ascending=False)

print("Детализированные результаты корреляции:")
print(corr_df)

# Проверка

Если эффект есть, значит он должен быть устойчивым.
Просто разделим данные на два промежутка. Выберем дату произвольно

In [ ]:
# Разделяем данные на две части по дате
split_date = '2019-01-01'

combined_before = combined.loc[:split_date]  # Все данные ДО 2019-01-01
combined_after = combined.loc[split_date:]   # Все данные ПОСЛЕ 2019-01-01 (включительно)

In [ ]:
combined_before.head(3)

In [ ]:
combined_after.head(3)

In [ ]:

# Создаем функцию для расчета корреляций
def calculate_correlations(df):
    corr_list = []
    for ticker in df.columns.drop('fluxadjflux'):
        r, p_value = pearsonr(df['fluxadjflux'], df[ticker])
        corr_list.append({
            'Ticker': ticker,
            'Correlation': r,
            'p-value': p_value,
            'Significant (5%)': p_value < 0.05
        })
    return pd.DataFrame(corr_list).sort_values('Correlation', key=abs, ascending=False)

# Считаем для первого периода
print("Корреляции ДО 2019-01-01:")
corr_before = calculate_correlations(combined_before)
print(corr_before)

# Считаем для второго периода
print("\nКорреляции ПОСЛЕ 2019-01-01:")
corr_after = calculate_correlations(combined_after)
print(corr_after)

**ВЫВОДЫ**: Эффект не повторился в двух разных промежутках времени


# Графики
все же попробуем графически понять почему некоторые тикеры  хорошо коррелируют

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_ticker_correlation(ticker, df=combined):
    """
    Анализирует корреляцию между солнечной активностью и указанным тикером.
    
    Параметры:
    ticker (str): Название тикера (например, 'XOM')
    df (DataFrame): DataFrame с данными (по умолчанию combined)
    """
    # Проверка наличия тикера
    if ticker not in df.columns:
        print(f"Ошибка: Тикер {ticker} отсутствует в данных")
        return
    
    # Создаем копию данных для безопасной обработки
    data = df[['fluxadjflux', ticker]].copy()
    
    # Нормализация данных
    def manual_minmax_scale(series):
        return (series - series.min()) / (series.max() - series.min())
    
    data_norm = pd.DataFrame({
        'fluxadjflux_norm': manual_minmax_scale(data['fluxadjflux']),
        f'{ticker}_norm': manual_minmax_scale(data[ticker])
    }, index=data.index)
    
    # График временных рядов
    plt.figure(figsize=(14, 6))
    plt.title(f'Сравнение нормализованных рядов: Солнечная активность vs {ticker}')
    
    plt.plot(data_norm['fluxadjflux_norm'], 
             label='Solar Flux (norm)', 
             color='orange',
             alpha=0.7)
    
    plt.plot(data_norm[f'{ticker}_norm'], 
             label=f'{ticker} Price (norm)', 
             color='blue',
             alpha=0.5,
             linestyle='--')
    
    plt.legend()
    plt.xlabel('Дата')
    plt.ylabel('Нормализованные значения')
    plt.grid(True)
    plt.show()
    
    # Scatter plot с регрессией
    plt.figure(figsize=(8, 6))
    sns.regplot(x=data['fluxadjflux'], 
                y=data[ticker],
                scatter_kws={'alpha':0.3, 'color':'green'},
                line_kws={'color':'red'})
    
    plt.title(f'Корреляция: Солнечный Flux vs Цена {ticker}')
    plt.xlabel('Solar Flux Adjusted')
    plt.ylabel(f'{ticker} Stock Price')
    plt.grid(True)
    
    # Рассчет корреляции
    r = data['fluxadjflux'].corr(data[ticker])
    plt.annotate(f'Pearson r = {r:.2f}\nN = {len(data):,}', 
                 xy=(0.05, 0.95), 
                 xycoords='axes fraction',
                 fontsize=12,
                 bbox=dict(facecolor='white', alpha=0.8))
    
    plt.show()


Нарисуем лучший случай

In [ ]:

analyze_ticker_correlation('XOM')

Нарисуем худший случай

In [ ]:
analyze_ticker_correlation('PFE')

# Выводы

- На Солнце есть длинные циклы.
- Иногда эти циклы совпадают с развитием компаний.
- Иногда эта корреляция довольно высокая